In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

In [2]:
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  x1,x2=x
  return 400*x1**2 + 0.004*x2**4

def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  x1,x2=x
  return np.array([800*x1, 0.016*x2**3])  

def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  x1,x2 = x
  return np.array([[800,0],[0,0.048*x2**2]])

In [3]:
def find_minimizer_newton(start_x, tol,step=1,backtrack=False):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  k = 0
  
  while np.linalg.norm(evalg(x)) > tol:
    D = np.linalg.inv(evalh(x))
    
    if backtrack:
      alp = compute_steplength_backtracking_scaled_direction(x,evalg(x),D,step,0.5,0.5)
    else:
      alp=step
    
    x =x - np.multiply(alp,np.matmul(D,evalg(x)))
    
   
    k+= 1
    if k%1000==0:
        print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')

    
  print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')
  return x,evalf(x),k

In [4]:
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  
  #Complete the code 
  alp = alpha_start
  while evalf(x - np.multiply(alp,gradf)) > evalf(x) - gamma*alp*np.matmul(gradf,gradf.T):
    alp *= rho
  return alp  

In [5]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  hes = evalh(x)
  vals = np.linalg.eigvals(hes)
  return np.array([[1/vals[0],0],[0,1/vals[1]]])



def compute_steplength_backtracking_scaled_direction(x, gradf, d, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(d) is np.ndarray and len(d) == 2 
  assert alpha_start>=0.
  assert type(rho) is float and rho>=0
  assert type(gamma) is float and gamma>=0

  alp = alpha_start
  x = np.array(x,dtype='float64')
  while evalf(x - np.multiply(alp,d@gradf)) > evalf(x) - gamma*alp*np.matmul(gradf,d@gradf.T):
    alp *= rho
  return alp


def find_minimizer_gdscaling(start_x, tol,alpha,rho,gamma):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  k = 0
  
  while np.linalg.norm(evalg(x)) > tol:
    D = compute_D_k(x)
    step = compute_steplength_backtracking_scaled_direction(x,evalg(x),D,alpha,rho,gamma)
    x = x - np.multiply(step,np.matmul(D,evalg(x)))
   
    k+= 1
    if k%1000==0:
        print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')

    
  print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')
  return x,evalf(x),k 

def find_minimizer_gd(start_x, tol,alpha, rho, gamma):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  
  x= start_x
  k = 0
 
  while np.linalg.norm(evalg(x)) > tol: 
    step = compute_steplength_backtracking(x,evalg(x),alpha,rho,gamma)
    x =x- np.multiply(step,evalg(x))
    k += 1
    if k%1000==0:
        print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')
    
  print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')

  return x, evalf(x),k  

In [6]:
X = np.array([2,2])
tol = 1e-9

x1,fx1,k1 = find_minimizer_newton(X,tol)

Iteration:16, Minimizer:[0.         0.00304488], min value:3.4382653805813626e-13, gradient norm:4.516786210993798e-10


In [7]:
x2, fx2, k2 = find_minimizer_newton(X,tol,backtrack=True)

Iteration:16, Minimizer:[0.         0.00304488], min value:3.4382653805813626e-13, gradient norm:4.516786210993798e-10


## 3)

In [8]:
print(f'Newtons method with constant steplength 1:   Minimizer : {x1}, Min function value:{fx2}, iterations: {k1}.\nNewtons method with backtracking line search: Minimizer: {x2}, Min function value: {fx2}, iterations {k2}.')

Newtons method with constant steplength 1:   Minimizer : [0.         0.00304488], Min function value:3.4382653805813626e-13, iterations: 16.
Newtons method with backtracking line search: Minimizer: [0.         0.00304488], Min function value: 3.4382653805813626e-13, iterations 16.


Actual minimizer : [0, 0]\
Min function value: 0

While using newton's method, we get the exact same minimizer and same number of iterations needed. This is because the optimal step size in each iteration turned out to be 1 (initial value) itself.
The error in minimizer is of the order 1e-3, while error in function value is of the order 1e-13

## 4)

In [9]:
x3, fx3, k3 = find_minimizer_gdscaling(X,tol,1,0.5,0.5)

Iteration:16, Minimizer:[0.         0.00304488], min value:3.4382653805813626e-13, gradient norm:4.516786210993798e-10


In [11]:
x4, fx4, k4 = find_minimizer_gd(X,tol,1,0.5,0.5)

Iteration:1000, Minimizer:[1.10291925e-04 1.65936666e+00], min value:0.030331864090665663, gradient norm:0.11458402212759414
Iteration:2000, Minimizer:[-3.40322605e-05  1.44810963e+00], min value:0.017590459803729492, gradient norm:0.05569548535885945
Iteration:3000, Minimizer:[-1.54276514e-05  1.30124667e+00], min value:0.011468381362291405, gradient norm:0.03735127818315328
Iteration:4000, Minimizer:[6.27150134e-05 1.19134186e+00], min value:0.008059171177036832, gradient norm:0.05700124397035226
Iteration:5000, Minimizer:[1.06226282e-05 1.10543147e+00], min value:0.005972973246628108, gradient norm:0.023223702959171918
Iteration:6000, Minimizer:[6.79718119e-06 1.03574144e+00], min value:0.0046032774472326, gradient norm:0.018590682163073496
Iteration:7000, Minimizer:[2.29438150e-05 9.77529341e-01], min value:0.003652617833822551, gradient norm:0.023670124906207003
Iteration:8000, Minimizer:[-3.48488758e-05  9.28257521e-01], min value:0.0029703315672528663, gradient norm:0.0306760569

KeyboardInterrupt: ignored

Since the hessian matrix is diagonal, it happens to coincide with our scaling matrix and hence we get the same answer for newtons method and GD with scaling\
Minimizer, min function value and errors are same as for newtons method.

GD without scaling is taking extremely long time to converge.